In [1]:
from utils.dataset_loader import dataset_loader


_, _, test_data_p = dataset_loader('cora')

In [5]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="dataset", name="Cora", split="full")
dataset = dataset[0]  # there is only one graph in the dataset

test_data = dataset.subgraph(dataset.test_mask)

for i in range(len(test_data.y)):
	print(test_data.y[i], test_data_p.y[i])

tensor(3) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(0) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(3) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(1) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2) tensor([0., 0., 0.])
tensor(2

In [1]:
from torch_geometric.datasets import Planetoid
import os
from torch_geometric.loader import DataLoader
import torch_geometric
import torch
import torch.nn.functional as F
from utils.dataset_loader import dataset_loader


DATASET_STORAGE_PATH = "./dataset/"

train_data, val_data, test_data = dataset_loader('cora')



In [16]:
# count the how many elements are for each class (are 1-hot encoded)
print(test_data.y[:,0].sum())
print(test_data.y[:,1].sum())
print(test_data.y[:,2].sum())
print(test_data.y[:,3].sum())


tensor(149.)
tensor(103.)
tensor(64.)
tensor(684.)


In [5]:
from utils.math import interval_softmax, actually_reachable
import torch
import numpy as np

a_L = np.array([9.6062404e-06, 9.9997389e-01, 1.6473108e-05])
a_U = np.array([9.6254726e-06, 9.9997395e-01, 1.6506086e-05])

assert np.all(a_L <= a_U), "Lower bound should be less than upper bound"

q_L, q_U = actually_reachable(a_L, a_U)

assert np.all(q_L <= q_U), "Lower bound should be less than upper bound"

print("Lower bound: ", q_L)
print("Upper bound: ", q_U)

Lower bound:  [9.6062404e-06 9.9997389e-01 1.6473108e-05]
Upper bound:  [9.62547260e-06 9.99973921e-01 1.65037596e-05]


In [1]:
from ogb.nodeproppred import NodePropPredDataset

In [29]:
ogb_dataset = NodePropPredDataset(name='ogbn-arxiv', root="dataset")

/vast/m.tolloso/miniconda3/envs/gu/lib/python3.11/site-packages/ogb/nodeproppred/dataset.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_dict = torch.load(pre_pr

In [38]:
ogb_dataset.graph # there is only one graph in the dataset

{'edge_index': array([[104447,  15858, 107156, ...,  45118,  45118,  45118],
        [ 13091,  47283,  69161, ..., 162473, 162537,  72717]]),
 'edge_feat': None,
 'node_feat': array([[-0.057943, -0.05253 , -0.072603, ...,  0.173364, -0.172796,
         -0.140059],
        [-0.1245  , -0.070665, -0.325202, ...,  0.068524, -0.372111,
         -0.301036],
        [-0.080242, -0.023328, -0.183787, ...,  0.109919,  0.117589,
         -0.139883],
        ...,
        [-0.22053 , -0.036568, -0.402199, ...,  0.11336 , -0.161393,
         -0.145171],
        [-0.138236,  0.040885, -0.251811, ..., -0.08929 , -0.041253,
         -0.376132],
        [-0.029875,  0.268417, -0.161124, ...,  0.120807,  0.077647,
         -0.091018]], dtype=float32),
 'node_year': array([[2013],
        [2015],
        [2014],
        ...,
        [2020],
        [2020],
        [2020]]),
 'num_nodes': 169343}

In [ ]:
import torch_geometric
import torch 
import numpy as np
from utils.math import even_quantile_labels
from torch_geometric.utils import subgraph

nclass = 5
train_ratio = 0.5
val_ratio = 0.2

edge_index = torch.as_tensor(ogb_dataset.graph['edge_index'])
node_feat = torch.as_tensor(ogb_dataset.graph['node_feat'])
label = even_quantile_labels(
        ogb_dataset.graph['node_year'].flatten(), nclass, verbose=False)

data = torch_geometric.data.Data(
	edge_index=edge_index,
	x=node_feat,
	y=label
)

num_nodes = data.num_nodes

# Step 1: Generate random permutation of nodes
indices = torch.randperm(num_nodes)
train_size = int(train_ratio * num_nodes)
val_size = int(val_ratio * num_nodes)

# Step 2: Split indices
train_idx = indices[:train_size]
val_idx = indices[train_size:train_size + val_size]
test_idx = indices[train_size + val_size:]

# Step 3: Create subgraphs based on the splits
train_edge_index, train_edge_attr = subgraph(train_idx, data.edge_index, data.edge_attr, relabel_nodes=True)
val_edge_index, val_edge_attr = subgraph(val_idx, data.edge_index, data.edge_attr, relabel_nodes=True)
test_edge_index, test_edge_attr = subgraph(test_idx, data.edge_index, data.edge_attr, relabel_nodes=True)

# Step 4: Create independent Data objects
train_data = torch_geometric.data.Data(
	x=data.x[train_idx],
	edge_index=train_edge_index,
	edge_attr=train_edge_attr,
	y=data.y[train_idx]
)

val_data = torch_geometric.data.Data(
	x=data.x[val_idx],
	edge_index=val_edge_index,
	edge_attr=val_edge_attr,
	y=data.y[val_idx]
)

test_data = torch_geometric.data.Data(
	x=data.x[test_idx],
	edge_index=test_edge_index,
	edge_attr=test_edge_attr,
	y=data.y[test_idx]
)




tensor(135474)
tensor(16934)
tensor(16935)


In [60]:
data.y

array([0, 1, 1, ..., 4, 4, 4])